In [ ]:
import pygame

# 1. 초기화 -> 변수선언
pygame.init()

# 2. 게임화면 설정 -> 크기 고정
size = [400, 900]
screen = pygame.display.set_mode(size)
title = "Using PyGame"

pygame.display.set_caption(title)

# 3. 게임 화면 내부 설정 -> 변수
clock = pygame.time.Clock()

class Object:
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def add_img(self, address):
        if address[-3:] == 'png':
            self.image = pygame.image.load(address).convert_alpha()
        else:
            self.image = pygame.image.load(address)
    
    def change_size(self, width, heigth):
        self.image = pygame.transform.scale(self.image, (width, heigth))
        self.width, self.height = self.image.get_size()
    
    def show(self):
        screen.blit(self.image, (self.x, self.y))

player = Object()
player.add_img("C:/Users/Administrator/Pictures/pygame/player.png")
player.change_size(50, 80)
player.x = round(size[0] / 2) - round(player.width / 2)
player.y = size[1] - player.height - 50

black = (0, 0, 0)

# player = pygame.image.load("C:/Users/Administrator/Pictures/pygame/player.png").convert_alpha()
# player = pygame.transform.scale(player, (50, 80))
# player_width, player_height = player.get_size()
# player_x = round(size[0] / 2) - round(player_width / 2)
# player_y = size[1] - player_height - 50

k = 0


# 4. 메인 이벤트
system_exit = 0

while(system_exit == 0):
    #  4 - 1) FPS(Frame Per Second) 설정
    clock.tick(60)
    
    #  4 - 2) 입력(키보드, 마우스) 감지
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            system_exit = 1
            
    #  4 - 3) 변화(입력 / 시간에 따른 변화)
    k += 1
    
        
    #  4 - 4) 전사작업(그리기)
    screen.fill(black)
#     screen.blit(player, (player_x, player_y))
    player.show()
    
    #  4 - 5) 업데이트
    pygame.display.flip()

# 5. 종료
pygame.quit()

pygame 2.4.0 (SDL 2.26.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
